# <center>A tutorial on 6D object pose estimation for AR application </center>


<div style="text-align: right"> Ikbeom Jeon (TA) (ikbeomjeon@kaist.ac.k) </div> 
<div style="text-align: right"> Vincent Lepetit (vincent.lepetit@enpc.fr) </div>  
<br>
<div style="text-align: right"> Last update: 14/07/2021</div>
    


### Description
We introduce~

### Citation
We have referenced the original code needed for writing this tutorial here.
https://github.com/ylabbe/cosypose

If you use the code in your research, please cite the paper:

```
@inproceedings{labbe2020,
title= {CosyPose: Consistent multi-view multi-object 6D pose estimation}
author={Y. {Labbe} and J. {Carpentier} and M. {Aubry} and J. {Sivic}},
booktitle={Proceedings of the European Conference on Computer Vision (ECCV)},
year={2020}}
```


## 0. Load required packages

In [ ]:
#%load_ext autoreload
#%autoreload 2

import pickle as pkl 
import numpy as np
import json
import pandas as pd
import torch
import torch.multiprocessing

from cosypose.lib3d import Transform
from cosypose.config import LOCAL_DATA_DIR
from cosypose.datasets.datasets_cfg import make_scene_dataset, make_object_dataset
import cosypose.utils.tensor_collection as tc

## 1. Load images and 3D models

In [ ]:
object_set = 'ycbv'

# Load dataset
scene_ds = make_scene_dataset('ycbv.test.keyframes')



## 2. Load results of 2D detection and initial 6D pose estimation from file



In [ ]:
def load_posecnn_results():
    results_path = LOCAL_DATA_DIR / 'saved_detections' / 'ycbv_posecnn.pkl'
    results = pkl.loads(results_path.read_bytes())
    infos, poses, bboxes = [], [], []

    l_offsets = (LOCAL_DATA_DIR / 'bop_datasets/ycbv' / 'offsets.txt').read_text().strip().split('\n')
    ycb_offsets = dict()
    for l_n in l_offsets:
        obj_id, offset = l_n[:2], l_n[3:]
        obj_id = int(obj_id)
        offset = np.array(json.loads(offset)) * 0.001
        ycb_offsets[obj_id] = offset

    def mat_from_qt(qt):
        wxyz = qt[:4].copy().tolist()
        xyzw = [*wxyz[1:], wxyz[0]]
        t = qt[4:].copy()
        return Transform(xyzw, t)

    for scene_view_str, result in results.items():
        scene_id, view_id = scene_view_str.split('/')
        scene_id, view_id = int(scene_id), int(view_id)
        n_dets = result['rois'].shape[0]
        for n in range(n_dets):
            obj_id = result['rois'][:, 1].astype(np.int)[n]
            label = f'obj_{obj_id:06d}'
            infos.append(dict(
                scene_id=scene_id,
                view_id=view_id,
                score=result['rois'][n, 1],
                label=label,
            ))
            bboxes.append(result['rois'][n, 2:6])
            pose = mat_from_qt(result['poses'][n])
            offset = ycb_offsets[obj_id]
            pose = pose * Transform((0, 0, 0, 1), offset).inverse()
            poses.append(pose.toHomogeneousMatrix())

    data = tc.PandasTensorCollection(
        infos=pd.DataFrame(infos),
        poses=torch.as_tensor(np.stack(poses)).float(),
        bboxes=torch.as_tensor(np.stack(bboxes)).float(),
    ).cpu()
    return data

posecnn_detections = load_posecnn_results()


You can visulize it in this way


## 3. Get better result using refinement network

In [ ]:
refiner_run_id = 'ycbv-refiner-finetune--251020'
n_coarse_iterations = 0
n_refiner_iterations = 2


## 4. 3D data augmentation





